In [1]:
import numpy as np
import pandas

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#from pymystem3 import Mystem
import pymorphy2

# инициализируем
vectorizer = CountVectorizer()
#m = Mystem()
morph = pymorphy2.MorphAnalyzer()

In [2]:
# функция для лемматизации

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = ""
    for word in words:
        p = morph.parse(word)[0]
        res = res + " " + p.normal_form

    return res

In [3]:
data_xls = pandas.read_excel("test_nlp.xlsx")
data_xls.head()

,Text,Priznak
0,Подготовить ответ,1
1,К исполнению,0
2,В работу,0
3,Подготовить служебную записку,1
4,Подготовить СЗ,1


In [4]:
# проводим лемматизацию
data_xls['Text'] = data_xls.apply(lambda row: lemmatize(row['Text']), axis=1)

In [5]:
data_xls.head()

,Text,Priznak
0,подготовить ответ,1
1,к исполнение,0
2,в работа,0
3,подготовить служебный записка,1
4,подготовить сз,1
5,в приказ,1
6,предоставить отчёт,1
7,для использование в работа,0


In [6]:
text = data_xls.Text

In [7]:
type(text)
text

0                 подготовить ответ
1                      к исполнение
2                          в работа
3     подготовить служебный записка
4                    подготовить сз
5                          в приказ
6                предоставить отчёт
7        для использование в работа
Name: Text, dtype: object

In [137]:
# тестовый вариант лемматизации на Mystem
text_test = "Съешь еще этих мягких французских булок да выпей чаю"
lemmas = m.lemmatize(text_test)
print(''.join(lemmas))

съедать еще этот мягкий французский булка да выпивать чай



In [138]:
# тестовый вариант лемматизации на pymorphy2

text_test = "Съешь еще этих мягких французских булок да выпей чаю"

words = text_test.split() # разбиваем текст на слова
res = ""
for word in words:
    p = morph.parse(word)[0]
    res = res + " " + p.normal_form

print (res)

 съесть ещё этот мягкий французский булка да выпить чай


In [8]:
# создаем векторы
text_vector = vectorizer.fit_transform(text)

In [9]:
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer, стоит отметить что порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']

['для',
 'записка',
 'исполнение',
 'использование',
 'ответ',
 'отчёт',
 'подготовить',
 'предоставить',
 'приказ',
 'работа',
 'сз',
 'служебный']

In [177]:
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('отчет') # вернет 2

In [10]:
# показать матрицу
text_vector_array = text_vector.toarray()
print(text_vector_array)

[[0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 1 0 0]]


In [11]:
past_columns = [] # пустой массив, для опрееделени колонок
for i in range(len(text_vector_array[0])): 
    past_columns.append(f"v_{i}")
past_columns

['v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5',
 'v_6',
 'v_7',
 'v_8',
 'v_9',
 'v_10',
 'v_11']

In [12]:
dataframe_vector = pandas.DataFrame(data = text_vector_array,columns = past_columns)
dataframe_vector

,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11
0,0,0,0,0,1,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,1,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,1,0
5,0,0,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,1,0,1,0,0,0,0
7,1,0,0,1,0,0,0,0,0,1,0,0


In [13]:
data = pandas.concat([data_xls,dataframe_vector],axis = 1)
data

,Text,Priznak,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11
0,подготовить ответ,1,0,0,0,0,1,0,1,0,0,0,0,0
1,к исполнение,0,0,0,1,0,0,0,0,0,0,0,0,0
2,в работа,0,0,0,0,0,0,0,0,0,0,1,0,0
3,подготовить служебный записка,1,0,1,0,0,0,0,1,0,0,0,0,1
4,подготовить сз,1,0,0,0,0,0,0,1,0,0,0,1,0
5,в приказ,1,0,0,0,0,0,0,0,0,1,0,0,0
6,предоставить отчёт,1,0,0,0,0,0,1,0,1,0,0,0,0
7,для использование в работа,0,1,0,0,1,0,0,0,0,0,1,0,0


In [14]:
Y = data.Priznak
X = data[past_columns]

In [15]:
# предсказание простое
clf = RandomForestClassifier(random_state=241)
clf.fit(X, Y)
# расчет веса показателей
importances = clf.feature_importances_
print(X.columns)
print(importances)

Index(['v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9',
       'v_10', 'v_11'],
      dtype='object')
[0.05967768 0.01831228 0.20741173 0.05500486 0.02930029 0.03401361
 0.15568189 0.05710884 0.06722708 0.25218659 0.04178814 0.02228701]


In [82]:
clf = RandomForestClassifier()

# здесь пишем какие параметры будет перебирать

 param_greed = {
    "max_depth" : [2,5,10],
    "criterion" : ['gini', 'entropy'],
    "min_samples_split" : [2,5,10],
    "min_samples_leaf" : [1,5,10],
}

In [85]:
# пишем каие параметры перебираем scoring = как оцениваем  ,
# cv = сколько тестовых выборок, n_jobs = -1 используем все процссоры
#GS = GridSearchCV(clf,param_greed,scoring = 'roc_auc',cv = 6,n_jobs = -1)
GS = GridSearchCV(clf,param_greed,scoring = 'roc_auc',cv = 6,n_jobs = -1)

In [86]:
GS.fit(X,Y)

ValueError: Invalid parameter splitter for estimator RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [16]:
# теперь можно быстро подсчитать вектор для нового документа
#text2 = ["the puppy"]
#vector = vectorizer.transform(text2)
#print(vector.toarray())

text_predict = 'работу'
text_predict_lemmat = lemmatize(text_predict)
new_v = vectorizer.transform([text_predict_lemmat])  # результат [[1 0 0 2]]
new_v_array = new_v.toarray()
dataframe_predict = pandas.DataFrame(data = new_v_array,columns = past_columns)
print(dataframe_predict)
clf.predict(dataframe_predict)

   v_0  v_1  v_2  v_3  v_4  v_5  v_6  v_7  v_8  v_9  v_10  v_11
0    0    0    0    0    0    0    0    0    0    1     0     0


array([0])